# Challenge 1

In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def is_running_on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ and os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive"
data_path = '/kaggle/input/' if is_running_on_kaggle() else 'data/'

#for dirname, _, filenames in os.walk(data_path):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

# Imports
import warnings
import matplotlib.pyplot as plt
from torchvision.io import read_image

class CactusDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [44]:
annotations_file = data_path + 'train.csv'
img_dir = data_path + 'train/train/'

BATCH_SIZE = 32
LEARNING_RATE = 1e-3

In [45]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class TinyModel(nn.Module):
#     def __init__(self):
#         super(TinyModel, self).__init__()
#         # Define convolutional layers
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
#         # Define fully connected layers
#         self.fc1 = nn.Linear(32 * 8 * 8, 128)  # 32x32 image downscaled by 2x2 max pooling twice
#         self.fc2 = nn.Linear(128, 1)  # Output layer with a single neuron for binary classification

#     def forward(self, x):
#         # Convolutional layers with ReLU activation and max pooling
#         x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)
#         x = F.relu(self.conv2(x))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)
#         # Flatten the output for fully connected layers
#         x = x.view(-1, 32 * 8 * 8)
#         # Fully connected layers with ReLU activation
#         x = F.relu(self.fc1(x))
#         # Output layer with sigmoid activation for binary classification
#         x = torch.sigmoid(self.fc2(x))
#         # x = x > 0.5
#         return x

# tinymodel = TinyModel()
# BATCH_SIZE = 32
# LEARNING_RATE = 1e-3

# print('The model: ', tinymodel)

# # Define the loss function
# loss_fn = torch.nn.BCELoss()

# # Define the optimizer
# optimizer = torch.optim.SGD(tinymodel.parameters(), lr=LEARNING_RATE)

# # Load the data
# train_dataloader = torch.utils.data.DataLoader(training_data, BATCH_SIZE, shuffle=True)

# # Train the model
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     for batch, (X, y) in enumerate(dataloader):
#         # Compute prediction and loss
#         pred = model(X.float())
#         print(pred)
#         print('Interval: ', pred.min(), pred.max())
#         y = y.reshape(-1, 1).float()
#         loss = loss_fn(pred, y)
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         if batch % 10 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# epochs = 5
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(train_dataloader, tinymodel, loss_fn, optimizer)

TODO: Try to preprocess like in ImProc

In [54]:
from torchvision.models import resnet18
from torchvision.transforms import Normalize
import torchvision.transforms as transforms

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ConvertImageDtype(torch.float32),
    transforms.Normalize(mean=mean, std=std),
])
training_dataset = CactusDataset(annotations_file, img_dir, transform=transform)

print(training_dataset[0][0].shape)

torch.Size([3, 224, 224])


In [56]:
from torch.utils.data import random_split

# Define the sizes of training and validation sets
train_size = 3200 # int(0.8 * len(training_dataset))
val_size = len(training_dataset) - train_size

# Split the dataset into training and validation sets
train_data, val_data = random_split(training_dataset, [train_size, val_size])

# Print the sizes of the training and validation sets
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))

Training set size: 3200
Validation set size: 14300


In [57]:
import torch.nn as nn

class ResnetClassificator(nn.Module):
    def __init__(self):
        super(ResnetClassificator, self).__init__()
        self.resnet = resnet18(pretrained=True)
        last_layer_size = self.resnet.fc.out_features # 1000        
        self.fc = nn.Linear(last_layer_size, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

In [60]:
model = ResnetClassificator()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_dataloader = torch.utils.data.DataLoader(training_dataset, BATCH_SIZE, shuffle=True)


for batch, (X, y) in enumerate(train_dataloader):
    # Compute prediction and loss
    pred = model(X.float())
    # print(pred)
    # print('Interval: ', pred.min(), pred.max())
    y = y.reshape(-1, 1).float()
    loss = criterion(pred, y)
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 10 == 0:
        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

In [50]:
# input_tensor = training_dataset[0][0]
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# model = ResnetClassificator()
# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)
# print(torch.argmax(probabilities))

tensor([-0.5271])
tensor([1.])
tensor(0)
